# Kubernetes: container orchestration

## Docker containers
* Program portability
* Take a program and all that it contains/needs (dependencies), put it in an image and run it isolated.

![docker](img/docker.png)

## Kubernetes
* You tell it what you want to run and it decides where to run it (nodes).
* The basic unit is the pod, usually with one container but can be more.
* Keeps things running and can scale the number of pods.
* Problems arise: networking, storage, how do containers find each other, load balancing.

![kube-architecture](img/kube-architecture.png)

## Architecture
* Master node(s): host the *control plane* that controls the whole kubernetes system (API server, scheduling, replication, node failure handling, etc).
* Worker nodes: they run the applications deployed.

![master-worker](img/master-worker.png)

## How do I run an application?
* Deployments: yaml manifests to tell kubernetes what you need to run.
* The deployment sets up a *replica set* that ensures the specified number of replicas (pods) is met.
* Old replica sets are stored and can be used for rollbacks.

```yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: snacker-deployment
spec:
  revisionHistoryLimit: 10
  replicas: 1
  selector:
    matchLabels:
      app: snacker
  template:
    metadata:
      labels:
        app: snacker
    spec:
      containers:
        - name: snacker
          image: registry.lambdaclass.com/snackerbis
          ports:
            - containerPort: 80
      imagePullSecrets:
        - name: lambdaclass-registry
```

![deployments](img/deployments.png)

## Exposing pods to the outside
* Because pods are ephemeral, communication to pods is done through *services*.
* Services are essentially pod load balancers for applications.
* To map specific hosts to services, we use an *ingress*.

![ingress](img/ingress.png)

```yaml
apiVersion: extensions/v1beta1
kind: Ingress
metadata:
  name: snackerbis-ingress
  annotations:
    kubernetes.io/ingress.class: "nginx"
  labels:
    app: snackerbis
spec:
  rules:
  - host: kube.lambdaclass.com
    http:
      paths:
      - path: /snackerbis
        backend:
          serviceName: snackerbis-service
          servicePort: 80
```